In [1]:
###FOR TTBAR BACKGROUND
####FOR Etau + TAU
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea import hist
from helpers import delta_r, delta_phi, inv_mass_3p, cos_opening_angle, inv_mass, delta_eta
from samples import signal_samples
from helpers import files_from_dir, files_from_dirs
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
from ROOT import TH1D
import warnings
from scipy.stats import chisquare
from scipy.stats import ks_2samp
import os
plt.style.use([hep.style.CMS])
%matplotlib inline
curr_os = os.name
local_dir = "/eos/user/l/lmollier/PDM/data/bck/nanoAOD/tt2l2nu"


workingpoint = 15
warnings.filterwarnings("ignore")
lumi = 60000. # pb-1
xsec = 87.315 #pb
scale = lumi * xsec
samples = files_from_dir(local_dir),
i_file = 1
tot_events = 0
file = samples[0][0]
print("PF and stuff like that")
events = NanoEventsFactory.from_root(file, schemaclass=NanoAODSchema).events()
events['SelElectron'] = events.Electron[(events.Electron.pt > 24.) & (events.Electron.mvaFall17V2Iso_WP90 > 0.5)]
events['SelMuon'] = events.Muon[(events.Muon.pt > 24.) & (events.Muon.mediumPromptId) & (events.Muon.pfRelIso03_all < 0.2) & (np.abs(events.Muon.dxy) < 0.005)]
muon1, electron1 = ak.unzip(ak.cartesian([events.SelMuon, events.SelElectron], nested=True))
match1 = ak.any(muon1.jetIdx == electron1.jetIdx, axis=-1, mask_identity=False)        
events['SelMuon'] = events.SelMuon[(~(match1))]    
events['SelTau'] = events.Tau[(events.Tau.pt > 20.) & (abs(events.Tau.eta) < 2.3)& (events.Tau.idDeepTau2017v2p1VSmu > 0.5) & (events.Tau.idDeepTau2017v2p1VSe > 0.5) & (events.Tau.idDeepTau2017v2p1VSjet >=7)]
tau2, electron2 = ak.unzip(ak.cartesian([events.SelTau, events.SelElectron], nested=True))
match2 = ak.any(tau2.jetIdx == electron2.jetIdx, axis=-1, mask_identity=False)
tau3, muon3 = ak.unzip(ak.cartesian([events.SelTau, events.SelMuon], nested=True))
match3 = ak.any(tau3.jetIdx == muon3.jetIdx, axis=-1, mask_identity=False)
events['SelTau'] = events.SelTau[((~(match2) & ~(match3)) )]  

events = events[ak.num(events.SelMuon) == 0]
events = events[ak.num(events.SelElectron) == 1]
events = events[ak.num(events.SelTau) == 2]
events['LeadTau'] = events.SelTau[:,0]
events['SubTau'] = events.SelTau[:,1]
events['LeadTau_pass'] = events.LeadTau[events.LeadTau.idDeepTau2017v2p1VSjet >=15]
events['LeadTau_fail'] = events.LeadTau[~(events.LeadTau.idDeepTau2017v2p1VSjet >=15)]
events['SubTau_pass'] = events.SubTau[events.SubTau.idDeepTau2017v2p1VSjet >=15]
events['SubTau_fail'] = events.SubTau[~(events.SubTau.idDeepTau2017v2p1VSjet >=15)]



#####
#binning for pt tau observable (analysis based on it)
#####
bins_tau_pt = hist.Bin("pT", "tau pT [GeV]",50, 20, 220)

#compute the prompt rate
# which is the rate at which prompt taus pass the selection
#first choose the prompt taus and then apply the selection, compute the ratio
print("Prompt rate")

events_pr = NanoEventsFactory.from_root(file, schemaclass=NanoAODSchema).events()
events_pr['SelElectron'] = events_pr.Electron[(events_pr.Electron.pt > 24.) & (events_pr.Electron.mvaFall17V2Iso_WP90 > 0.5)]
events_pr['SelMuon'] = events_pr.Muon[(events_pr.Muon.pt > 24.) & (events_pr.Muon.mediumPromptId) & (events_pr.Muon.pfRelIso03_all < 0.2) & (np.abs(events_pr.Muon.dxy) < 0.005)]
muon1, electron1 = ak.unzip(ak.cartesian([events_pr.SelMuon, events_pr.SelElectron], nested=True))
match1 = ak.any(muon1.jetIdx == electron1.jetIdx, axis=-1, mask_identity=False)        
events_pr['SelMuon'] = events_pr.SelMuon[(~(match1))]  
events_pr['SelTau'] = events_pr.Tau[(events_pr.Tau.pt > 20.) & (abs(events_pr.Tau.eta) < 2.3)& (events_pr.Tau.idDeepTau2017v2p1VSmu > 0.5) & (events_pr.Tau.idDeepTau2017v2p1VSe > 0.5) & (events_pr.Tau.idDeepTau2017v2p1VSjet >=7)]
tau2, electron2 = ak.unzip(ak.cartesian([events_pr.SelTau, events_pr.SelElectron], nested=True))
match2 = ak.any(tau2.jetIdx == electron2.jetIdx, axis=-1, mask_identity=False)
tau3, muon3 = ak.unzip(ak.cartesian([events_pr.SelTau, events_pr.SelMuon], nested=True))
match3 = ak.any(tau3.jetIdx == muon3.jetIdx, axis=-1, mask_identity=False)
events_pr['SelTau'] = events_pr.SelTau[((~(match2) & ~(match3)) )]  
prompt_before_sel = ak.flatten(events_pr.SelTau.pt)

# h_prompt_bef = hist.Hist("prompt taus before selection",
#                      bins_tau_pt
#                      )
# h_prompt_bef.fill(pT=prompt_before_sel)
h_prompt_bef = TH1D("prompt taus before selection","prompt taus before selection", 50, 20, 220)

#after selection
events_pr['SelTau'] = events_pr.SelTau[events_pr.SelTau.idDeepTau2017v2p1VSjet >=15]
prompt_after_sel = ak.flatten(events_pr.SelTau.pt)
h_prompt_aft = hist.Hist("prompt taus after selection",
                     bins_tau_pt
                     )
h_prompt_aft.fill(pT=prompt_after_sel)
h_prompt_rate = hist.plotratio(num = h_prompt_aft, denom = h_prompt_bef)

###compute the fake rate
### with pt tau
### in the CR region (one bjet) and before and after ID tau selection
print("fake rate")

events_fr = NanoEventsFactory.from_root(file, schemaclass=NanoAODSchema).events()
events_fr['SelElectron'] = events_fr.Electron[(events_fr.Electron.pt > 24.) & (events_fr.Electron.mvaFall17V2Iso_WP90 > 0.5)]
events_fr['SelMuon'] = events_fr.Muon[(events_fr.Muon.pt > 24.) & (events_fr.Muon.mediumPromptId) & (events_fr.Muon.pfRelIso03_all < 0.2) & (np.abs(events_fr.Muon.dxy) < 0.005)]
muon1, electron1 = ak.unzip(ak.cartesian([events_fr.SelMuon, events_fr.SelElectron], nested=True))
match1 = ak.any(muon1.jetIdx == electron1.jetIdx, axis=-1, mask_identity=False)        
events_fr['SelMuon'] = events_fr.SelMuon[(~(match1))]    
events_fr['SelTau'] = events_fr.Tau[(events_fr.Tau.pt > 20.) & (abs(events_fr.Tau.eta) < 2.3)& (events_fr.Tau.idDeepTau2017v2p1VSmu > 0.5) & (events_fr.Tau.idDeepTau2017v2p1VSe > 0.5) & (events_fr.Tau.idDeepTau2017v2p1VSjet >=7)]
tau2, electron2 = ak.unzip(ak.cartesian([events_fr.SelTau, events_fr.SelElectron], nested=True))
match2 = ak.any(tau2.jetIdx == electron2.jetIdx, axis=-1, mask_identity=False)
tau3, muon3 = ak.unzip(ak.cartesian([events_fr.SelTau, events_fr.SelMuon], nested=True))
match3 = ak.any(tau3.jetIdx == muon3.jetIdx, axis=-1, mask_identity=False)
events_fr['SelTau'] = events_fr.SelTau[((~(match2) & ~(match3)) )]  

events_fr = events_fr[ak.num(events_fr.SelMuon) == 1]
events_fr = events_fr[ak.num(events_fr.SelElectron) == 1]
jets = events_fr.Jet[events_fr.Jet.pt > 25.]
bjets = jets[jets.btagDeepFlavB > 0.2770]
one_bjet = ak.num(bjets) == 1
no_bjet = ak.num(bjets) == 0

CR_loose = events_fr[one_bjet]
CR_medium = events_fr[one_bjet]
CR_medium['SelTau'] = CR_medium.SelTau[CR_medium.SelTau.idDeepTau2017v2p1VSjet >=workingpoint]

CR_loose = CR_loose[ak.num(CR_loose.SelTau) == 1]
CR_medium = CR_medium[ak.num(CR_medium.SelTau) == 1]
CR_loose_kin = ak.flatten(CR_loose.SelTau.pt)
CR_medium_kin = ak.flatten(CR_medium.SelTau.pt)
h_fakerate_bef = hist.Hist("fake rate taus before selection",
                     bins_tau_pt
                     )
h_fakerate_bef.fill(pT=CR_loose_kin)
h_fakerate_aft = hist.Hist("fake rate taus after selection",
                     bins_tau_pt
                     )
h_fakerate_aft.fill(pT=CR_medium_kin)
h_fake_rate = hist.plotratio(num = h_fakerate_aft, denom = h_fakerate_bef)



ModuleNotFoundError: No module named 'ROOT'